In [ ]:
# importing of libaries
%matplotlib inline

import requests
import kaggle
import os 
import json

import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
# connecting to api, and downloading data set (checking it works as well)
dataset = "prasad22/healthcare-dataset"
kaggle.api.dataset_download_files(dataset, path="datasets", unzip=True)

print(os.listdir("datasets"))

file_path = "datasets/healthcare_dataset.csv"
df = pd.read_csv(file_path)

print(df.head())
print(df.info())

In [ ]:
#cleaning of data
df.columns = df.columns.str.capitalize().str.replace(' ', '_')
#print(df.columns)
df['Name'] = df['Name'].str.title()
df['Gender'] = df['Gender'].str.capitalize()
df['Blood_type'] = df['Blood_type'].str.upper()
df['Hospital'] = df['Hospital'].str.title()
df['Doctor'] = df['Doctor'].str.title()
df['Insurance_provider'] = df['Insurance_provider'].str.title()
df['Billing_amount'] = df['Billing_amount'].round(2)
df['Date_of_admission'] = pd.to_datetime(df['Date_of_admission'])
df['Discharge_date'] = pd.to_datetime(df['Discharge_date'])

#check for nulls / empty cells which there is not 
print(df.isnull().sum())

#checking for negatives/outliers
print(df['Billing_amount'].describe())
# There is a negative billing amount, so I will look into that more later
print(df['Age'].describe())
print(df['Gender'].value_counts())
print(df['Blood_type'].value_counts())
print(df['Admission_type'].value_counts())

print(df.head())

In [ ]:
#querying negatives
negatives = df[df['Billing_amount'] < 0]
#print(negatives)

#investigating if it's a particular medical condition / hospital
print(f"Negative billing entries: {len(negatives)}")
print(negatives['Medical_condition'].value_counts())
print(negatives['Hospital'].value_counts())

#creating a refund column for reporting later on 
df['Billing_flag'] = df['Billing_amount'].apply(lambda x: 'Refund' if x < 0 else 'Charge')
df['Net_billing'] = df['Billing_amount']

print(df['Billing_flag'].value_counts())
print(df.groupby('Billing_flag')['Billing_amount'].sum())

In [ ]:
# checking for & removing dupkicates  
duplicates = df.duplicated().sum()
#print(f'Duplicates: {duplicates}')
# Get all duplicated rows (all columns identical), sort by name and display to check they are true dupliates
duplicates = df[df.duplicated(keep=False)]
duplicates_sorted = duplicates.sort_values(by='Name')
#print(duplicates_sorted.head(10))
# drop duplicates
df = df.drop_duplicates()

In [ ]:
#Now the data is cleaned, im starting to visualise it to identify trends
#looking at the breakdown of age and billing to identify trends visually

df['Age'].hist(bins=20)
plt.title("Age Distribution")
plt.show()

df['Billing_amount'].hist(bins=20)
plt.title("Billing Amount Distribution")
plt.show()

sns.countplot(x='Gender', data=df)
plt.show()


#looking at length of stay and cost per day and adding a column for it
df['Date_of_admission'] = pd.to_datetime(df['Date_of_admission'])
df['Discharge_date'] = pd.to_datetime(df['Discharge_date'])

df['Length_of_stay'] = (df['Discharge_date'] - df['Date_of_admission']).dt.days
df['Cost_per_day'] = df['Billing_amount'] / df['Length_of_stay'].replace(0, 1)

print(df.head())

In [ ]:
#exploring data and looking for further correlations
numeric_df = df.select_dtypes(include=['int64', 'float64'])
#print(numeric_df.head())

corr = numeric_df.corr()
#print(corr)

plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Correlation Heatmap")
plt.show()

sns.boxplot(x="Blood_type", y="Billing_amount", data=df)
plt.title("Billing Amount by Blood Type")
plt.show()

sns.boxplot(x="Admission_type", y="Billing_amount", data=df)
plt.title("Billing Amount by Admission Type")
plt.show()

#df.groupby("Hospital")["Billing_amount"].mean().sort_values(ascending=False).head(10)
print (df.head())

In [ ]:
# Define the categorical variables I wanted to analyse and visualise 
categorical_vars = ["Gender", "Blood_type", "Medical_condition"]

for var in categorical_vars:
    #print(f"\n===== {var} vs Billing_amount =====\n")
    
    # Grouped summary statistics
    #summary = df.groupby(var)["Billing_amount"].agg(['mean', 'median', 'count']).sort_values(by="mean", ascending=False)
    #print(summary.head(10))  # show top 10 if there are many categories
    
    # Boxplot for distribution comparison
    plt.figure(figsize=(10,5))
    sns.boxplot(x=var, y="Billing_amount", data=df)
    plt.title(f"Billing Amount by {var}")
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
#savng to visulise in PowerBI 
save_path = r"C:\Users\James's Laptop\OneDrive\Data Camp\Projects\Healthcare_Proj"
file_path = os.path.join(save_path, "healthcare_cleaned.csv")
df.to_csv(file_path, index=False)
print(os.getcwd())
print("Export complete: healthcare_cleaned.csv")